**INTRODUCTION**

The research of video games has become an increasingly important field with researchers trying to understand player behavior, engagement, and decision-making. Games like Minecraft provide a rich environment for studying how players interact with virtual worlds, what motivates them to continue playing, and how they engage with game-related content outside the game itself.

One key aspect of player engagement that is being investigated in this project is whether individuals choose to subscribe to game-related newsletters, which may provide updates, community news, or exclusive content. Subscription behavior can be influenced by multiple factors, including a player's in-game activity, demographics, and personal interest in the game. Identifying which characteristics are most predictive of subscription can help game developers and researchers better target recruitment efforts, optimize outreach strategies, and enhance player retention.
This study focuses on data collected from a Minecraft research server operated by a group at the University of British Columbia (UBC). Their mission is to advance the frontiers of artificial intelligence through groundbreaking work in generative modelling, reinforcement learning, probabilistic programming, and Bayesian inference. Players' actions within the game are logged, allowing researchers to analyze patterns in behavior and demographic traits. In this project, our main characteristics are age and the hours played by a user and seeing how they connect to their subscription status.

Understanding these predictive factors can have practical implications for managing server resources, tailoring communication strategies, and improving the overall player experience.

The research question that we are aiming to explore in this project is: Can age of player and hours played predict whether or not a player is subscribed to a game-related newsletter in the players.csv dataset? This involves using a knn-classification on the data to predict either a TRUE or FALSE in player subscription. 

The dataset that we'll be using for our prediction is "players.csv", which contains 196 observations and 7 variables:
1. experience
     - the player's previous experiences in minecraft
2. subscribe
     - whether the player is subscribed to a game-related newsletter
3. hashedEmail
     - the player's email in an encrypted format
4. played_hours
     - the amount of hours the player has played on Minecraft
5. name
     - the player's name
6. gender
     - the player's recorded gender
7. age
     - recorded age of the player
  
There may be some discrepencies in the data with actual facts, as the data is completely reliant on accurate and truthful reporting of data when it comes to each player. There is also no guarantee that the players were all completely honest about their details.

**METHODS & RESULTS**

We plan to determine whether the variables are individually good predictors of subscription, then use the good ones together in a model. We will use k-nearest neighbours classification since it's versatile and I'm predicting a non-numeric variable.

We will split 75% of the data for training. Then we will cross-validate using a V of 5, and find the most accurate k value from 1 to 196. After that we will find the tested accuracy of the optimal k then see if it's good.

A variable will be a good predictor with a testing accuracy above 55% (the model performs better than random guessing, give or take the standard 5% uncertainty). We want the same or better for our conjunction model.

This model will use knn, so we'll be sure to scale and center the data. We are assuming that the data (both self-reported and virtually recorded) is accurate, and that the sample is representative enough.

In [ ]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 10)
source("cleanup.R")

In [ ]:
set.seed(42)

In [ ]:
#reading in the data
players_csv <- read_csv("https://raw.githubusercontent.com/nothingbutash/dsci-100-2024w2-group-006-2/refs/heads/main/players.csv")

In [ ]:
players <- players_csv |>
    #Changing column names to be more uniform (removed camel case and capitalization)
    #Also had to make subscribe a factor rather than logical, as classification does not work otherwise
    mutate(experience = as_factor(experience), hashed_email = hashedEmail, age = Age, subscribe = as_factor(subscribe)) |>
    select(-hashedEmail, -Age) |>
    na.omit()
players

In [ ]:
sample_stats <- players |>
    #calculating the mean, median, and standard deviation with summarize
    summarize(sample_mean = mean(age, na.rm = TRUE), sample_med = median(age, na.rm = TRUE), sample_sd = sd(age, na.rm = TRUE))
sample_stats

sample_distribution <- ggplot(players, aes(x = age)) + 
   geom_histogram(binwidth = 1) +
   labs(x = "Age (Years)", y = "Number of People") +
   ggtitle("Age Distribution of Players")
sample_distribution

In [ ]:
hours_distribution<-ggplot(players,aes(x = played_hours))+
geom_histogram(bins = 10,fill = "blue")+
labs(x = "Played hours", y = "Count")+
ggtitle("Distribution of played hours")
hours_distribution

In [ ]:
subscribed_players <- players |>
    filter(subscribe == TRUE) |>
    nrow()
nonsubscribed_players <- players |>
    filter(subscribe == FALSE) |>
    nrow()
print(paste0(subscribed_players, " players out of 196 are subscribed, aka ", subscribed_players/196*100, " percent."))


**(3)**
Above, we loaded and wrangled the dataset. The data appears tidy (one observation per row, value per cell, and variable per column).

The table of mean values is below:

In [ ]:
#It was a fairly simple process to get a table of the numeric variable's mean values, as I applied the process learned in class
mean_table <- players |>
    select(played_hours,age) |>
    map_df(mean, na.rm = TRUE)
# noticed that there's at least one NA value in Age (???) so I had to remove them
mean_table

The subscription percentages appear to be quite similar. This may not be a good predictor (or be problematic to analyze). However, prior to further predictive analysis this is unproven.

In [ ]:
#For this visualization, I plan to plot age against playtime, with colour denoting subscription status (similar to the cancer classification example from class)
playtime_age_plot <- players |>
    ggplot(aes(x = age, y = played_hours, color = subscribe)) +
    geom_point(alpha = 0.6) +
    labs(x = "Age (years)", y = "Playtime (Hours)", color = "Subscribed (yes or no)", title = "Age of player vs. average hours played in game")

playtime_age_plot

It appears most players have playtimes below 10 hours, and anyone above that is subscribed, which is intriguing. Additionally, higher playtimes are more common among younger people (age > 30), emphasizing the importance of scaling and centering the data.

In [ ]:
#cut out extreme outliers to provide better visual of data points
playtime_age_plot_better_visual <- playtime_age_plot +
    ylim(0, 4.5) +
    labs(title = "Age of player vs. average hours played in game (edited)")
playtime_age_plot_better_visual


To start off, we will analyze whether age is a good predictor of subscription status. To do this, we will split the data with 75% going to training and 25% going to testing, with the strata being the age.

In [ ]:
players_split <- initial_split(players, prop = 0.75, strata = subscribe)  
players_train <- training(players_split)   
players_test <- testing(players_split)

Made a recipe for predicting subscribe based on age.

In [ ]:
players_recipe <- recipe(subscribe ~ age + played_hours, data = players_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) 


Made the model specification, a knn model for classification.

In [ ]:
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
      set_engine("kknn") |>
      set_mode("classification")

Made the workflow for tuning the model, collected the metrics, and found the best value of k.

In [ ]:
players_vfold <- players_train |>
    vfold_cv(v = 5, strata = subscribe)

k_vals <- tibble(neighbors = seq(from = 1, to = 112, by = 1))

knn_results <- workflow() |>
      add_recipe(players_recipe) |>
      add_model(knn_tune) |>
      tune_grid(resamples = players_vfold, grid = k_vals) |>
      collect_metrics()
knn_results



Now we will get the best k value in terms of accuracy. This can also be plotted as a visualization.

In [ ]:
best_k <- knn_results |>
    filter(.metric == "accuracy") |>
    slice_max(order_by = mean, n = 1)
best_k

best_k_plot <- knn_results |>
    filter(.metric == "accuracy") |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "K value (1 to 28)", y = "Percent Accuracy of Model", title = "K Values vs Accuracy")
best_k_plot

From this data, the best k value appears to be k = 16

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 16) |>
      set_engine("kknn") |>
      set_mode("classification")

In [ ]:
players_fit <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(knn_spec) |>
    fit(data = players_train)

Now we need to get the accuracy of the model on the testing set.

In [ ]:
model_accuracy <- predict(players_fit, players_test) |>
    bind_cols(players_test) |>
    metrics(truth = subscribe, estimate = .pred_class) |>
    head(1) |>
    pull()
model_accuracy

Our accuracy with the model trained on age, is 62.3%. This is greater than 55%, thus age and played hours are a good predictor of subscription status.

**DISCUSSION**

The scatter plot shows the relationship between player age and playtime, distinguishing between subscribed and unsubscribed users. The majority of players seem to be under 30 years old, with a playtime of under 10 hours. A few outliers, which we cut out in the second graph for better visualization of the other points, were seen with over 150 hours of play time, and a few above 25 hours, all below 30 years of age. Because of this, we can observe that the general trend indicates that younger players tend to have higher playtime, particularly between the ages of 10 and 30 years. Subscription status also appears to be more common among younger players with there being a larger cluster amongst those who have lower playtime. After accounting for outliers and extreme values, it would be fair to conclude that players are more likely to be subscribed if they are younger and have a lower amount of playtime hours. However, this correlation is quite weak, as the points frequently overlap and are relatively spread out throughout the graph. 

This aligns with expectations, since younger players likely have more free time for gaming. Those who play less could also mean they are just starting to cultivate an interest in the game, and would be more inclined to subscribe due to their lack of experience. As for the outliers, the extremely high play times may suggest a more dedicated and experienced player, who, as seen in the data, would be subscribed due to their high engagement. 

With the knowledge of this trend, game developers can appropriately tailor subscription models and marketing strategies to target younger players to further capitalize on their current demographic or potentially find ways to address the older demographic and convince them to subscribe. 

A good question to ponder for the future would be whether other variables like experience could affect the subscription status as it is possible that the outliers might be veteran players who have high engagement with the game. Similarly, we theorized that since the bulk of subscriptions come from younger players with low playtime that they might be Beginner players, therefore it would be interesting to see whether these theories would be right and whether they have an actual impact. To improve, the study can also aim to increase its sample size and recruit more players to answer the survey, as it would account for greater variability and allow for a more accurate trend to be observed. 

Last but not least, our accuracy of the final model is about 65.3% when the neighbour is equal to 20, which is greater than 55% which shows it is a good predictor. These results align with general expectations for k-NN models. When k is small, the model tends to be more sensitive to noise and may overfit the training data. As k increases, the model becomes more generalized by considering more neighbors, leading to better performance up to a point. The observed plateau around k = 20 to k = 21 reflects the typical behavior of k-NN. Thus, this finding is consistent with theoretical expectations and demonstrates a good example of model tuning in action.

Since, selecting the appropriate k value is crucial for optimizing model performance, the fact that accuracy improves significantly with the chosen k value suggests that model tuning should never be overlooked. In real-world applications, a 10-20% improvement in accuracy can significantly influence outcomes.

Furthermore, understanding the plateau behavior helps prevent unnecessary computational expense—there’s little to gain from using extremely high k values once optimal performance has been reached. In large datasets, higher k values increase computational load, so identifying the optimal range ensures better efficiency.

Overall, this investigation explored the extent to which two key factors in player age and hours played can predict whether a player has subscribed to a game-related newsletter.

Our analysis suggests that both age and hours played have meaningful, though not exhaustive, relationships with subscription behavior. Younger players and those who are relatively new on the server are more likely to subscribe to the newsletter, indicating that active engagement and demographic factors play a role in a player’s willingness to stay connected with the game's ecosystem. Conversely, older players with low playing time show lower subscription rates, possibly reflecting either a more casual approach to gaming or differing interests in supplemental content.

However, while the predictive power of these two features is useful, it is also limited. Player behavior is shaped by a wide range of social, motivational, and contextual factors, many of which go beyond simple quantitative metrics like age and playtime. To build more developed predictive models in the future, it would be beneficial to include additional behavioral data, such as variables like experience and gender. This may yield deeper insights into why some players engage more deeply with a game's community than others.

From a practical perspective, these findings offer actionable insights for the UBC research group. Understanding the characteristics of likely subscribers can help inform how and when to prompt players to sign up for the newsletter.

Ultimately, this project illustrates the potential of data science to bridge gameplay behavior with real-world decisions and thereby contributing to both the success of research projects and the richness of the player experience and engagement.
